In [114]:

class Term:
    def __str__(self):
        pass

    def debug_str(self):
        pass

    def is_var(self):
        return False
    
    def is_lambda(self):
        return False

    def is_app(self):
        return False

    def is_lambdaDB(self):
        return False

class Var(Term):
    def __init__(self, var):
        self.var = var

    def __str__(self):
        return str(self.var)

    def is_var(self):
        return True

    def debug_str(self):
        return f"Var {self.var}"

class Lambda(Term):
    def __init__(self, var: str, t: Term):
        self.var = var
        self.t = t

    def __str__(self):
        return f"λ{self.var}. {self.t}"

    def debug_str(self):
        return f"Lambda {self.var}. ({self.t.debug_str()})"

    def is_lambda(self):
        return True

class LambdaDB(Term):
    def __init__(self, t: Term):
        self.t = t

    def __str__(self):
        return f"λ. {str(self.t)}"

    def debug_str(self):
        return f"Lambda . ({self.t.debug_str()})"

    def is_lambdaDB(self):
        return True

    def is_lambda(self):
        return True


class App(Term):
    def __init__(self, t1: Term, t2: Term):
        self.t1 = t1
        self.t2 = t2

    def __str__(self):
        # return f"({str(self.t1)}) ({str(self.t2)})"
        res = ""
        if (self.t1.is_var()):
            res += f"{self.t1}"
        else: res += f"({self.t1})"

        if (self.t2.is_var()):
            res += f" {self.t2}"
        else:
            res += f" ({self.t2})"

        return res

    def debug_str(self):
        return f"App ({self.t1.debug_str()}), ({self.t2.debug_str()})"

    def is_app(self):
        return True


In [97]:
import re

def is_var(token):
    return re.match("^[a-z]+$", token)

def is_lambda(token):
    return re.match("^lambda [a-z]+[.]$", token)

def is_term(token):
    return isinstance(token, Term)

def lambda_var(token):
    return token[7:-1]

def parse_brackets(stack):
    res = None
    while (len(stack) > 0 and stack[-1] != "("):
        token = stack.pop()
        
        if (is_term(token)):
            if res == None:
                res = token
            else:
                res = App(token, res)
        elif is_var(token):
            if res == None:
                res = Var(token)
            else:
                res = App(Var(token), res)
        elif is_lambda(token):
            lam_var = lambda_var(token)
            res = Lambda(lam_var, res)

    if (len(stack) > 0):
        stack.pop()

    stack.append(res)

    return stack

def parse_term(term_str: str):
    term_str = ' '.join(term_str.split())

    regex = "lambda [a-z]+[.]|[a-z]+|[(]|[)]"

    tokens = re.findall(regex, term_str)
    stack = []
    print(tokens)
    for token in tokens:
        # print(list(map(str, stack)))
        if token == ")":
            stack = parse_brackets(stack)
        elif is_var(token):
            if len(stack) == 0:
                stack.append(Var(token))
            elif is_term(stack[-1]):
                t = stack.pop()
                app = App(t, Var(token))
                stack.append(app)
            elif is_var(stack[-1]):
                var = stack.pop()
                app = App(Var(var), Var(token))
                stack.append(app)
            else:
                stack.append(token)
        else:
            stack.append(token)


        while len(stack) >= 2 and is_term(stack[-1]) and is_term(stack[-2]):
            t2 = stack.pop()
            t1 = stack.pop()

            stack.append(App(t1, t2))

    stack = parse_brackets(stack)

    return stack[0]


t = parse_term("a b c d")

print(t.debug_str())

['a', 'b', 'c', 'd']
App (App (App (Var a), (Var b)), (Var c)), (Var d)


In [98]:
def deBruijn(term: Term):
    free_vars = {}
    def convert(t: Term, bound_vars):
        if t.is_var():
            try:
                return Var(bound_vars.index(t.var))
            except:
                if t.var not in free_vars:
                    free_var_index = len(free_vars)
                    free_vars[t.var] = free_var_index
                return Var(free_vars[t.var] + len(bound_vars))
        elif t.is_app():
            return App(convert(t.t1, bound_vars), convert(t.t2, bound_vars))
        elif t.is_lambda():
            return LambdaDB(convert(t.t, [t.var] + bound_vars))

    return convert(term, []), free_vars


term = parse_term("lambda k. k x y z")
db_term, free_vars = deBruijn(term)
print(db_term)
print(free_vars)



['lambda k.', 'k', 'x', 'y', 'z']
λ. ((0 1) 2) 3
{'x': 0, 'y': 1, 'z': 2}


In [99]:
def is_free_var(var):
    return isinstance(var, str)


def eval_krivine(term: Term):
    stack = []
    env = []

    while True:
        # print(term, list(map(str, stack)), (list(map(str, env))))
        print(term)
        if term.is_app():
            stack.append((term.t2, env))
            term = term.t1

        elif term.is_lambdaDB():
            if len(stack) == 0:
                return term, env

            stack_top = stack.pop()
            env = [stack_top] + env
            term = term.t

        elif term.is_var():
            if is_free_var(term.var):
                if len(stack) == 0:
                    return term, env
                else:
                    raise Exception("Not implemented")
            else:
                index = term.var
                term, env = env[index]


In [141]:
# [n -> s]t

def substitution(t, n, s,):
    if t.is_var():
        if t.var == n:
            return s
        else:
            return t
    elif t.is_app():
        return App(subst(t.t1, n, s), subst(t.t2, n, s))
    elif t.is_lambdaDB():
        return LambdaDB(subst(t.t, n+1, s))


def normalize_subst_cbn(t):
    if t.is_app():
        t1_norm = normalize_subst_cbn(t.t1)
        if t1_norm.is_lambdaDB():
            beta_reduced = substitution(t1_norm.t, 0, t.t2)
            return normalize_subst_cbn(beta_reduced)
        return App(t1_norm, t.t2)

    return t

def normalize_subst_no(t):
    t_normalized_cbn = normalize_subst_cbn(t)

    if t_normalized_cbn.is_lambdaDB():
        return LambdaDB(normalize_subst_no(t_normalized_cbn.t))
    elif t_normalized_cbn.is_app():
        return App(normalize_subst_no(t_normalized_cbn.t1), normalize_subst_no(t_normalized_cbn.t2))

    return t_normalized_cbn


t = parse_term("a (lambda x. (lambda y. lambda z. y z) (lambda z. z) x)")
dbt, frees = deBruijn(t)
print(t)
print(dbt)
print(dbt.debug_str())
print(frees)
print("------------------")
print(normalize_subst_cbn(dbt))
print(normalize_subst_no(dbt))


['a', '(', 'lambda x.', '(', 'lambda y.', 'lambda z.', 'y', 'z', ')', '(', 'lambda z.', 'z', ')', 'x', ')']
a (λx. ((λy. λz. y z) (λz. z)) x)
0 (λ. ((λ. λ. 1 0) (λ. 0)) 0)
App (Var 0), (Lambda . (App (App (Lambda . (Lambda . (App (Var 1), (Var 0)))), (Lambda . (Var 0))), (Var 0)))
{'a': 0}
------------------
0 (λ. ((λ. λ. 1 0) (λ. 0)) 0)
0 (λ. 0)


In [101]:
term = parse_term("lambda x. y x")
t, frees = deBruijn(term)
print(t)

s, f = deBruijn(parse_term("lambda x.x x x"))

# redukcja t s = (lambda. y 0) (lambda. 0 0 0)


def beta_reduction(t, s):
    return LambdaDB(subst(t.t, 0, s))

print(beta_reduction(t,s))


['lambda x.', 'y', 'x']
λ. 1 0
['lambda x.', 'x', 'x', 'x']
λ. 1 (λ. (0 0) 0)


In [ ]:
class Nat(Term):
    def __init__(self, n):
        self.n = n

class Add(Term):
    def __init__(self, n, m):
        self.n = n
        self.m = m

class Mul(Term):
    def __init__(self, n, m):
        self.n = n
        self.m = m

class Sub(Term):
    def __init__(self, n, m):
        self.n = n
        self.m = m

class Eq(Term):
    def __init__(self, n, m):
        self.n = n
        self.m = m

class Tru(Term):
    pass

class Fls(Term):
    pass

class If(Term):
    def __init__(self, cond, if_true, if_false):
        self.cond = cond
        self.if_true = if_true
        self.if_false = if_false

class Fix(Term):
    def __init__(self, f):
        self.f = f

class Pair(Term):
    def __init__(self, first, second):
        self.first = first
        self.second = second

class Fst(Term):
    def __init__(self, pair):
        self.pair = pair

class Snd(Term):
    def __init__(self, pair):
        self.pair = pair

class Nil(Term):
    pass

class Cons(Term):
    def __init__(self, head, tail):
        self.head = head
        self.tail = tail

class Head(Term):
    def __init__(self, t):
        self.list = t

class Tail(Term):
    def __init__(self, t):
        self.list = t

class IsNil(Term):
    def __init__(self, t):
        self.list = t
